In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import (
    InMemoryConversationMemory,
    ToolConfig,
    ToolParameter,
    ToolProperty,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now set for a comfortable party mood! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are dimmed to create the perfect ambiance. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=305, completion_tokens=117, total_tokens=422), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_SSUxEBTvdl4EIJTqIJ3TrZiK', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_XvpLda4Q6FfaCPq3wN8GP5bS', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_8O4HRosMMSVEGmqZ3P7MRfIG', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The place is now set for a comfortable party mood! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are dimmed to create the perfect ambiance. Enjoy the party!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 305,
  'completion_tokens': 117},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_SSUxEBTvdl4EIJTqIJ3TrZiK',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_XvpLda4Q6FfaCPq3wN8GP5bS',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_8O4HRosMMSVEGmqZ3P7MRfIG',
    

Usage in the response object is total usage of the usage to call tool and the usage to use tool.

In [10]:
response.usage

Usage(prompt_tokens=305, completion_tokens=117, total_tokens=422)

Asynchronous generation

In [11]:
response = await chat_openai.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now set for a lively party atmosphere! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are perfectly dimmed to enhance the mood. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=305, completion_tokens=117, total_tokens=422), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_ZfyGnBeQEIlcQHlI29Xs8G37', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_62yHPbzzwiy09KhcwnbTzEDL', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_W7RLyUyKxB8Z88rBfbv1BZfQ', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type

multiple generation

In [12]:
response = chat_openai.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The place is now set for a comfortable party mood! The disco ball is spinning, the music is pumping at 120 BPM, and the lights are dimmed to create the perfect atmosphere. Enjoy the party!'},
  {'text': 'The party mood is set! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are dimmed to create the perfect ambiance. Enjoy the party! 🎉🕺💃'}],
 'name': None}

You can get the results from specific tool.

In [13]:
response = chat_openai.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've started some energetic and loud music at 120 BPM to get the party vibes going! What else can I do to enhance the party mood?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 244,
  'completion_tokens': 46},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_1QB8y7Nj16WlagvJWXNK4e1C',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_1QB8y7Nj16WlagvJWXNK4e1C',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=True, bpm='120'"}]}

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [14]:
response = chat_openai.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_r9Zp46M0EXqnZv8GDgH5Sx3n', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_R6UVWKW18zgNMP1oNVW4HFRw', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_yycMrupS8NqxgpnR3nw7u8X1', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_r9Zp46M0EXqnZv8GDgH5Sx3n'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_R

In [15]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 171,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_r9Zp46M0EXqnZv8GDgH5Sx3n',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_R6UVWKW18zgNMP1oNVW4HFRw',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_yycMrupS8NqxgpnR3nw7u8X1',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_r9Zp46M0EXqnZv8GDgH5Sx3n'},
   

In [16]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_r9Zp46M0EXqnZv8GDgH5Sx3n', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_R6UVWKW18zgNMP1oNVW4HFRw', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_yycMrupS8NqxgpnR3nw7u8X1', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [17]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_r9Zp46M0EXqnZv8GDgH5Sx3n'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_R6UVWKW18zgNMP1oNVW4HFRw'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_yycMrupS8NqxgpnR3nw7u8X1')], name=None)

Streaming

In [18]:
response = chat_openai.stream(prompt)

# Response is CompletionResults object
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextC

In [19]:
response = chat_openai.astream(prompt)

# Response is CompletionResults object
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The disco ball is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextC

# Pydantic tools

You can define tools as pydantic models. To use pydantic model as tool, make sure the following points: 

- Write description of top level docstring field in the class. This is used for tool description.
- Define each argument fileds with description
- Implement run() method.

Tool definition using `ToolProperty`, `ToolParameter` and `ToolConfig` is almost equivalent to the following way. 

In [20]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field


class PowerDiscoBall(BaseModel):
    """Powers the spinning disco ball."""

    power: bool = Field(description="Boolean to spin disco ball.")

    def run(self) -> str:
        return f"Disco ball is {'spinning!' if self.power else 'stopped.'}"


class DimLights(BaseModel):
    """Dim the lights."""

    brightness: float = Field(description="The brightness of the lights, 0.0 is off, 1.0 is full.")

    def run(self) -> str:
        return f"Lights are now set to {self.brightness}"


class BPM(Enum):
    """The beats per minute of the music."""

    SLOW = "60"
    MEDIUM = "120"
    FAST = "180"


class StartMusic(BaseModel):
    """Play some music matching the specified parameters."""

    energetic: bool = Field(description="Whether the music is energetic or not.")
    loud: bool = Field(description="Whether the music is loud or not.")
    bpm: BPM = Field(description="The beats per minute of the music.")
    # bpm: Literal["60", "120", "180"] = Field(..., description="The beats per minute of the music.") # This also works

    def run(self) -> str:
        return f"Starting music! {self.energetic=} {self.loud=}, {self.bpm=}"

In [21]:
# Convert to ToolConfig

tools = [PowerDiscoBall, StartMusic, DimLights]
tool_configs_pydantic = [ToolConfig.from_pydantic(tool) for tool in tools]

In [22]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_configs=tool_configs_pydantic,
)

In [23]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect atmosphere. Enjoy the celebration! 🎉💃✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 320,
  'completion_tokens': 119},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_EXtnUaNnAyGhXTVLVzrZv1BD',
     'function': {'arguments': '{"power": true}', 'name': 'PowerDiscoBall'},
     'type': 'function'},
    {'id': 'call_VxP5nKmzSD9f6er8uhFwzyaj',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'StartMusic'},
     'type': 'function'},
    {'id': 'call_Sule4fHn1iUOe8gSG99MBnBM',
     'func

### Array property for OpenAI API

Function Calling with array property for OpenAI Chat Completion.

In [24]:
def get_ingredients(ingredients: list[str]) -> str:
    """Prepare ingredients to make a meal."""
    return f"Preparing {ingredients}."


tool_configs_array = [
    ToolConfig(
        name="get_ingredients",
        description="Prepare ingredients to make a meal.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="ingredients",
                    type="array",
                    description="List of ingredients.",
                    items={"type": "string"},
                ),
            ],
            required=["ingredients"],
        ),
    ),
]

In [25]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[get_ingredients],
    tool_configs=tool_configs_array,
)

In [26]:
response = chat_openai.run(
    prompt="What are the ingredients to make omelet rice?",
    tool_choice="get_ingredients",
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "To make omelet rice, you'll need the following ingredients:\n\n- Eggs\n- Cooked rice\n- Butter\n- Onion\n- Bell pepper\n- Ham or chicken (your choice of protein)\n- Soy sauce\n- Ketchup\n- Salt\n- Pepper\n\nThese ingredients come together to create a delicious and satisfying dish!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 187,
  'completion_tokens': 101},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'What are the ingredients to make omelet rice?'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_s6CnNXqtKI93Phzrn6x37SFO',
     'function': {'arguments': '{"ingredients":["eggs","cooked rice","butter","onion","bell pepper","ham or chicken","soy sauce","ketchup","salt","pepper"]}',
      'name': 'get_ingredients'},
     'type': 'function'}]},
  {'role': 'tool',
   'too

### Azure OpenAI Chat Completion 

In [27]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [28]:
response = chat_openai_azure.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The disco ball is spinning, the music is pumping, and the lights are perfectly dimmed. The party mood is officially on! Enjoy! 🎉🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 305,
  'completion_tokens': 110},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_u69URSYieSjmk12ouWW2wBvL',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_ZGJ8tEcYTnqIO2LBC9JlsUdq',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_kxXBkfqwYS92vmWDWQbgGvvD',
     'function': {'arguments': '{"brightness": 0.3}', 'name': 'dim

In [29]:
response = await chat_openai_azure.arun(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The disco ball is spinning, the energetic music is pumping at 120 BPM, and the lights are dimmed to set the perfect party atmosphere. Let’s get this party started! 🕺💃✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 305,
  'completion_tokens': 117},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_RLu1CXct3IkTcNDTYwH7nbz6',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_Ng3zIOLAnYmashipdzDFt8Gr',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_nSDSTgPRQ7zlhpKQhbCOWCSW',
     'function': {'arguments'

In [30]:
response = chat_openai_azure.run(prompt, tool_choice="start_music")

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Got the music started! Let's also add some colorful lighting, comfortable seating, and some tasty snacks to set the party mood. How does that sound?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 244,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_tbZjZ0mk2LsnzaJIpTTZsWFJ',
     'function': {'arguments': '{"energetic":true,"loud":false,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_tbZjZ0mk2LsnzaJIpTTZsWFJ',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=False, bpm='120'"}]}

In [31]:
response = chat_openai_azure.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextC

In [32]:
response = chat_openai_azure.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextCont

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [33]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [34]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, some energetic music playing, and I've dimmed the lights. Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 91},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }


Asynchronous generation

In [35]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, fired up the disco ball, and put on some high-energy tunes! Let's get this party started! What else can I do to set the mood? 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 101},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
     

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [36]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm starting the music! Let's get this party going! 🎉 🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 276,
  'completion_tokens': 45},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! energetic=True loud=True, bpm=\'120\'"
        

Also pydantic tools is acceptable.

In [37]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [38]:
response = gemini.run(prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, cranked up some energetic tunes, and even got the disco ball going! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 87},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "DimLights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
  

Streaming

In [39]:
response = gemini.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='Alright')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, cranked up some energetic tunes, and")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, cranked up some energetic tunes, and got the disco ball spinning! Let's get this party started! 🎉 ")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, cranked up some energetic tunes, and got the disco ball spinning! Let's get this party started! 🎉 \n")], name=None), usage=Usage(prompt_tokens=0, comple

In [40]:
response = gemini.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='Alright')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music! Let's get this party started! 🎉 \n")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Alright, I've dimmed the lights, turned on the disco ball, and started some upbeat music! Let's get this party started! 🎉 \n")], name=None), usage=Usage(prompt_tokens=353, completion_tokens=8

### Gemini on VertexAI

In [41]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [42]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I've got this!  Let's get this party started! \n\n* **Music:** I've queued up some energetic tunes with a BPM of 120. It's a good beat to get everyone moving, but not too loud to have a conversation. \n* **Lights:** The lights are dimmed to a comfortable level, creating a nice, cozy atmosphere. \n* **Disco Ball:** The disco ball is spinning, adding a touch of fun and sparkle to the party!\n\nI hope you have a great time!  Let me know if you need anything else!  🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 181,
  'completion_tokens': 149},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
         

multiple generation

In [43]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Okay, I've got this! To create a comfortable party mood, I've instructed the tools to:\n\n* **Start the music:**  I've asked for music with a **medium tempo (120 bpm)**, something upbeat and energetic but not too loud.  \n* **Dim the lights:** I've set the lights to a **soft, cozy brightness (50%)**. This will create a relaxed and inviting atmosphere.\n* **Turn on the disco ball:** The disco ball will add a touch of fun and whimsy, creating a cool and festive vibe. \n\nLet's get this party started! 🎉 \n"},
  {'text': "Okay, I've got this! To create a comfortable party mood, I've instructed the tools to:\n\n* **Start the music!** I've chosen a beat with a bpm of 120. It's upbeat but not too intense, so the vibe stays relaxed. \n* **Dim the lights.**  A little less brightness helps set a more intimate and cozy atmosphere. \n* **Turn on the disco ball!** A bit of sparkle always adds a fun, festive touch.\n\nNow, just add some snacks and drinks,

Pydantic tool use

In [44]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [45]:
response = gemini_vertexai.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Okay, I\'ve started some upbeat music with a medium tempo, dimmed the lights a bit, and turned on the disco ball!  \n\nIs there anything else I can do to help get the party started?  Maybe:\n\n* **Set the temperature**: "Set the thermostat to 72 degrees Fahrenheit." \n* **Open the windows**: "Open the windows to let in some fresh air." \n* **Add some snacks**: "Order some pizza and snacks."\n* **Light some candles**: "Light some scented candles." \n\nLet me know what you\'d like to do next! \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 181,
  'completion_tokens': 143},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: 

### Anthropic Claude

tool_configs can be shared with Claude.

In [46]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [47]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a warm and inviting atmosphere without being too dark.\n2. Started playing energetic music at a moderate volume (not too loud) with a tempo of 120 beats per minute. This should provide a good rhythm for a party without overwhelming conversation.\n3. Turned on the disco ball to add some fun, moving lights to the space.\n\nThis combination should create a lively yet comfortable party environment. The dimmed lights and spinning disco ball will create an interesting visual atmosphere, while the energetic but not too loud music will provide a good backdrop for socializing and having fun.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1566,
  'completion

Asynchronous generation

In [48]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and relaxed atmosphere without being too dark.\n2. Started some music that's not too energetic or loud, with a moderate tempo of 120 beats per minute. This should provide a nice background ambiance without overwhelming conversation.\n3. Turned on the disco ball to add a touch of fun and visual interest to the room.\n\nThese settings should create a comfortable party environment that's perfect for socializing and enjoying the company of others. The dimmed lights and gentle music will encourage conversation, while the spinning disco ball adds a festive touch without being too intense.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable? For example, we could change the music tempo or adjust the lighting further if you prefer."}],
  'name': No

Pydantic tool use

In [49]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [50]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started some energetic music at a moderate volume. The music has a medium tempo of 120 beats per minute, which is perfect for a comfortable party vibe - not too slow, but not too fast either.\n3. Turned on the disco ball to add some fun, sparkling lights to the room.\n\nThese settings should create a welcoming and lively atmosphere that's still comfortable for socializing. The combination of soft lighting, upbeat but not overpowering music, and the subtle sparkle from the disco ball should set the perfect mood for your party.\n\nIs there anything else you'd like to adjust? Maybe you want the music a bit louder, the lights brighter or dimmer, or perhaps you'd prefer to turn off the disco ball? Feel free to let me know if you want

### Claude on Amazon Bedrock

In [51]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [52]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 30% (0.3) of full intensity. This creates a cozy and inviting atmosphere without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party mood up, but set it to not be too loud so people can still comfortably chat. The beats per minute (BPM) is set to 120, which is a moderate tempo that's good for both background music and dancing if people feel like it.\n\n3. Activated the disco ball: I've turned on the spinning disco ball to add a fun, party-like ambiance to the room. The soft, moving lights from the disco ball will complement the dimmed room lights nicely.\n\nThese settings should create a comfortable party mood that's lively enough for fun but still allows for conversation and relaxation. Is there anything else you'd like me to adjust to make the atmosphere even better?

In [53]:
repsonse = await claude.arun(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 30% (0.3) of full intensity. This creates a cozy and inviting atmosphere without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party mood up, but set it to not be too loud so people can still comfortably chat. The beats per minute (BPM) is set to 120, which is a moderate tempo that's good for both background music and dancing if people feel like it.\n\n3. Activated the disco ball: I've turned on the spinning disco ball to add a fun, party-like ambiance to the room. The soft, moving lights from the disco ball will complement the dimmed room lights nicely.\n\nThese settings should create a comfortable party mood that's lively enough for fun but still allows for conversation and relaxation. Is there anything else you'd like me to adjust to make the atmosphere even better?

Pydantic tool use

In [54]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [55]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a warm and inviting atmosphere without being too dark.\n2. Started playing music that's not too energetic or loud, with a moderate tempo of 120 BPM. This should provide a nice background ambiance for conversation and light dancing if desired.\n3. Turned on the disco ball to add a touch of fun and visual interest to the room.\n\nThese settings should create a comfortable party mood that's perfect for socializing and enjoying the company of others. The dimmed lights and moderate music allow for easy conversation, while the disco ball adds a festive touch without overwhelming the space.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable? For example, we could change the music tempo or adjust the lighting if you prefer. Just let me know!"}],
  'name'

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [56]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None, type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None, type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lig

Then these client tool config objects is transformed to client tools by format() method.

In [57]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'required': ['power'],
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}}}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [58]:
from langrila.gemini.genai import GeminiToolConfig

In [59]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParame

In [60]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For OpenAI Chat Completion

In [61]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [62]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The atmosphere is transforming! The lights are dimmed to a cozy 50% brightness, the disco ball is spinning, and energetic music with a lively beat is pumping through the air. Get the party started! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 306,
  'completion_tokens': 124},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_QK72mHGY08mYzO9ARKO6RXLE',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_Uuwdqa042S8FpoB3q9suK2oM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_rFZJOORAEPMVdbvmV8otl7YB',
     'function': {'argume

Next turn

In [63]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The music has been cranked up! The BPM is now at a thrilling 180, setting the perfect energetic vibe for your party. Let’s keep the energy high! 🎶✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 605,
  'completion_tokens': 63},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_QK72mHGY08mYzO9ARKO6RXLE',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_Uuwdqa042S8FpoB3q9suK2oM',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_rFZJOORAEPMVdbvmV8otl7YB',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}'

### For Gemini

In [64]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [65]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning,  pumped up the music, and dimmed the lights.  Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 353,
  'completion_tokens': 91},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
  

Tool calling is not needed additionally if the thing is already known.

In [66]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current music is playing at **120 bpm**.  Is that a good groove, or should I crank it up/down a bit? 🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 574,
  'completion_tokens': 46},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  

## Multi-turn conversation using tools with multiple model

The universal message and tool system allows us to chat multiple model across client.

In [67]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [68]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! The disco ball is spinning, the energetic music is playing loud at 120 BPM, and the lights are dimmed to create a cozy atmosphere. Get ready to have some fun! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 305,
  'completion_tokens': 124},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_CaUXQxBVRO9v7RwUDCESk40Q',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_ogH9pu0ks73vPCM8ueSZE6Vr',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_fYkK3TCf98igyozj7cMBKwpO',
     'f

Claude can continue conversation without any additional process.

In [69]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The music has now been changed to a faster tempo with 180 BPM. This should create a more intense and lively atmosphere for your party. The music is still energetic and loud, but now with a much faster beat that's perfect for dancing and keeping the energy high.\n\nIs there anything else you'd like to adjust to perfect the party mood?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 2062,
  'completion_tokens': 227},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_CaUXQxBVRO9v7RwUDCESk40Q',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_ogH9pu0ks73vPCM8ueSZE6Vr',
     'type': 'tool_use',
     'input': {'energetic': True, 'loud': True, 'bpm': '120'},
     'name': 'start_music'},
    {'id':

Go back to OpenAI Chat Completion

In [70]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music has been switched to a more relaxing vibe, now playing at 60 BPM with a soft and mellow tone. It's perfect for winding down and creating a chill atmosphere. If you need any further adjustments, just let me know! 🎶🌙"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 875,
  'completion_tokens': 77},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_CaUXQxBVRO9v7RwUDCESk40Q',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_ogH9pu0ks73vPCM8ueSZE6Vr',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': '

Gemini can follow as well

In [71]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music is now back to a comfortable 120 BPM. It's a nice, balanced tempo. Let me know if you'd like to make any more adjustments to the music or the overall atmosphere! 😊 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 1290,
  'completion_tokens': 70},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
  

In [72]:
prompt = "OK, then turn brightness to 0.7."

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The lights have been adjusted to a brightness level of 0.7. This should create a slightly brighter atmosphere than before, while still maintaining a comfortable ambiance for your party.\n\nTo summarize the current party setup:\n1. The disco ball is spinning\n2. The music is playing at 120 BPM, not too energetic or loud\n3. The lights are now set to a brightness of 0.7\n\nIs there anything else you'd like to change or adjust to perfect the party mood?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 3275,
  'completion_tokens': 182},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_CaUXQxBVRO9v7RwUDCESk40Q',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_ogH9pu0ks73vPCM8ueSZE6Vr',
     'type': 't

In [73]:
prompt = "Finally stop the disco ball."

response = gemini.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The disco ball has been switched off. The party atmosphere is now more relaxed and laid-back with the dimmed lights and comfortable music. \n\nFeel free to request any other adjustments to the music, lighting, or anything else to enhance the ambiance.  Enjoy your party! 🎉 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 1845,
  'completion_tokens': 72},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
          

If you input non-relavant topic, any tool is not called.

In [74]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I don't have real-time weather information or internet access to look it up for you. However, you can easily check the current weather by using a weather app, website, or asking a voice assistant on your device. If you let me know your location, I can provide general information on the typical weather patterns for this time of year!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 1813,
  'completion_tokens': 106},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_CaUXQxBVRO9v7RwUDCESk40Q',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_ogH9pu0ks73vPCM8ueSZE6Vr',
     'function': {'arguments': '{"ener

Conversation is recorded with serialized universal message format.

In [75]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_CaUXQxBVRO9v7RwUDCESk40Q'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_ogH9pu0ks73vPCM8ueSZE6Vr'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_fYkK3TCf98igyozj7cMBKwpO'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_CaUXQxBVRO9v7RwUDCESk40Q',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_ogH9pu0ks73vPCM8ueSZE6Vr',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_fYkK3TC